# 1. Some Graph Terminology

<p align="center">
<img src='Figs/Figure1.1.png' width=500>
</p>

- A graph $G$ consists of a vertex set $V$ and an edge set $E$.
- $G = (V, E)$
- the size of the vertex set = $\mid V \mid$
- the size of the edge set = $\mid E \mid$
- an edge is written as an ordered pair consisting of the vertices connected by the edge. 
- the ordered pair $(u, v)$ indicates the edge that connects vertex $u$ to vertex $v$.

$$
\begin{aligned}
V &= {a, b, c, d, e} \\
E &= {(a, b), (a, d), (b, d), (c, a), (c, e), (d, c), (d, e)} \\
G &= (V, E)
\end{aligned}
$$

- a graph can be directed or undirected
- an edge of a directed graph is an ordered pair $(u, v)$ with $u$ as the source vertex and $v$ as the target vertex.
- an undirected graph an edge, $(u, v)$ and $(v, u)$ are the same edge

- an edge $(u, v)$, then vertex $v$ is said to be adjacent to vertex $u$.
$$
\begin{aligned}
Adjacent[a] &= {b, d} \\
Adjacent[b] &= {d} \\
Adjacent[c] &= {a, e} \\
Adjacent[d] &= {c, e} \\
Adjacent[e] &= \{\}
\end{aligned}
$$

- For a directed graph, edge $(u, v)$ is an out-edge of vertex $u$ and an in-edge of vertex $v$.

$$
\begin{aligned}
OutEdges[a] &= {(a, b), (a, d)} \\
OutEdges[b] &= {(b, d)} \\
OutEdges[c] &= {(c, a), (c, e)} \\
OutEdges[d] &= {(d, c), (d, e)} \\
OutEdges[e] &= \{\}
\end{aligned}
$$

$$
\begin{aligned}
InEdges[a] &= {(c, a)} \\
InEdges[b] &= {(a, b)} \\
InEdges[c] &= {(d, c)} \\
InEdges[d] &= {(a, d), (b, d)} \\
InEdges[e] &= {(c, e), (d, e)}
\end{aligned}
$$

# 2. Graph Concepts
## 2.1. Vertex and Edge Descriptors

- vertices and edges are manipulated through opaque handles called *vertex descriptors* and *edge descriptors*
- different graph types may use different types for their descriptors
- the descriptor types for a graph type are always accessible through ***graph\_traits*** class
- The following function template 2 shows an implementation a generic function that determines if an edge is a self-loop.

**Code:**
```C++
template <typename Graph>
bool is_self_loop(typename graph_traits<Graph>::edge descriptor e, const Graph& g)
{
    typename graph_traits<Graph>::vertex_descriptor u, v;
    u = source(e, g);
    v = target(e, g);
    return u == v;
}
```

## 2.2. Property Maps

Graphs become useful as models for particular problem domains by associating objects and
quantities to vertices and edges. Figure 1.1 each vertex has a name consist-
ing of a single character, and each edge has a transmission delay.

**Property**

- attached objects or attached quantities
- property is used as a data members of a struct, separate arrays indexed by vertex or edge number,
hash tables, and so on

**Property Maps**
- an object that provides a mapping from a set of key objects to a set
of value objects
- The property map concepts specify only three functions: 
    - *get(p map, key)*: returns the value object for the *key*
    - *put(p map, key, value)*: assigns the value to the value object associated with the *key*
    - *p\_map[key]*: returns a reference to the value object

**Code: Prints the name of a vertex given a name property map**

```C++
template <typename VertexDescriptor, typename VertexNameMap>
void print vertex name(VertexDescriptor v, VertexNameMap name map)
{
    std::cout << get(name map, v);
}
```

**Code: Similarly, the transmission delay of an edge can be printed using the following function**
```C++
template <typename Graph, typename TransDelayMap, typename VertexNameMap>
void print_trans_delay(typename graph_traits<Graph>::edge_descriptor e, const Graph& g, 
    TransDelayMap delay_map, VertexNameMap name_map)
{
    std::cout << "trans-delay(" << get(name map, source(e, g)) << ","
    << get(name map, target(e, g)) << ") = " << get(delay map, e);
}
```

## 2.3. Graph Traversal



There are five kinds of graph iterators, one for each kind of collection:

1. A ***vertex iterator*** is used to traverse all the vertices of a graph. The value type of a
vertex iterator is a vertex descriptor.
2. An ***edge iterator*** is used to traverse all the edges of a graph. The value type of this
iterator is an edge descriptor.
3. An ***out-edge iterator*** is used to access all of the out-edges for a given vertex $u$. Its value type is an edge descriptor. Each edge descriptor in this iterator range will have $u$ as the source vertex and a vertex adjacent to $u$ as the target vertex (regardless of whether the graph is directed or undirected).
4. An ***in-edge iterator*** is used to access the in-edges of a vertex $v$. Its value type is an edge descriptor. Each edge descriptor in this iterator range will have $v$ as the target vertex and a vertex that $v$ is adjacent to as the source.
5. An ***adjacency iterator*** is used to provide access to the vertices adjacent to a given vertex. The value type of this iterator is a vertex descriptor.

Like descriptors, each graph type has its own iterator types that are accessible through
the ***graph_traits*** class

**Prints the names of all of the vertices in a graph**
```C++
template <typename Graph, typename VertexNameMap>
void print_vertex_names(const Graph& g, VertexNameMap name_map)
{
    std::cout << "vertices(g) = { ";
    typedef typename graph_traits<Graph>::vertex_iterator iter_t;
    for (std::pair<iter_t, iter_t> p = vertices(g);  p.first != p.second; ++p.first) {
        print vertex_name(*p.first, name_map); std::cout << ’ ’;
    }
std::cout << "}" << std::endl;
}
```

**Output:**
```C++
vertices(g) = { a b c d e }
```

**Prints the transmission delay values that are attached to each of the edges
in the graph** 

Note: In this function we use the ***tie()*** function (from ***boost/tuple/tuple.hpp*** ) to allow direct assignment from a ***std::pair*** into two scalar variables—in this case, first and last.

```C++
template <typename Graph, typename TransDelayMap, typename VertexNameMap>
void print trans delays(const Graph& g, TransDelayMap trans_delay_map,
VertexNameMap name_map)
{
    typename graph_traits<Graph>::edge_iterator first, last;
    for (tie(first, last) = edges(g); first != last; ++first) {
        print trans_delay(*first, g, trans_delay_map, name_map);
        std::cout << std::endl;
    }
}
```

**Output:**
```C++
trans−delay(a,b) = 1.2
trans−delay(a,d) = 4.5
trans−delay(b,d) = 1.8
trans−delay(c,a) = 2.6
trans−delay(c,e) = 5.2
trans−delay(d,c) = 0.4
trans−delay(d,e) = 3.3
```

there are ***out_edges()*** , ***in_edges()*** , and ***adjacent_vertices()*** functions. These functions take a vertex descriptor and graph object as arguments and return a pair of iterators.

## 2.4 Graph Construction and Modification

Defines interfaces for adding and removing vertices and edges from a graph.

- ***add_vertex()***: 
    - add the five nodes representing routers to the graph
    - returns a vertex descriptor for the new vertex (*we use this vertex descriptor to assign a vertex name to the vertex in a name property map*)
- ***add_edge()***: 
    - add edges representing connections between the routers
    - returns a ***std::pair***, where the first member of the pair is an edge descriptor for the new edge and the second is a Boolean flag that indicates whether an edge was added
    - some graph types will not insert an edge if an edge with the same source and target is already in the graph

**Code: Consturction**
```C++
template <typename Graph, typename VertexNameMap, typename TransDelayMap>
void build_router_network(Graph& g, VertexNameMap name_map,
TransDelayMap delay_map)
{
    <Add routers to the network 9a>
    <Add connections to the network 9b>
}
```

**Code: Add routers to the network 9a**
```C++
    typename graph traits<Graph>::vertex descriptor a, b, c, d, e;
    a = add vertex(g); name_map[a] = ’a’;
    b = add vertex(g); name_map[b] = ’b’;
    c = add vertex(g); name_map[c] = ’c’;
    d = add vertex(g); name_map[d] = ’d’;
    e = add vertex(g); name_map[e] = ’e’;
```

**code: Add connections to the network 9b**
```C++
    typename graph_traits<Graph>::edge_descriptor ed;
    bool inserted;

    tie(ed, inserted) = add_edge(a, b, g);
    delay map[ed] = 1.2;
    tie(ed, inserted) = add_edge(a, d, g);
    delay map[ed] = 4.5;
    tie(ed, inserted) = add_edge(b, d, g);
    delay map[ed] = 1.8;
    tie(ed, inserted) = add_edge(c, a, g);
    delay map[ed] = 2.6;
    tie(ed, inserted) = add_edge(c, e, g);
    delay map[ed] = 5.2;
    tie(ed, inserted) = add_edge(d, c, g);
    delay map[ed] = 0.4;
    tie(ed, inserted) = add_edge(d, e, g);
    delay map[ed] = 3.3;
```

In some cases it is more efficient to add or remove multiple vertices or edges simulta-
neously instead of one at a time. The BGL interface includes functions for accomplishing
this.

## 2.5. Algorithm Visitors

